# Exploration of Raw Waiting Time Data

This notebook is used to explore the raw waiting time data to determine what processing 
should take place. It does not incorporate information from other sources like weather
or holidays

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
import calendar

sns.set(rc={'figure.figsize':(16,9)})

In [ ]:
DATA_PATH = "wartezeiten_app211005.csv"
df = pd.read_csv(DATA_PATH, index_col="Unnamed: 0")

df["date"] = df.datum.str.slice(0, len("2020-01-01"))
df["time"] = df.datum.str.slice(len("2020-01-01 "))
df["time_min"] = df.time.str.slice(0, len("00:00"))
df.rename(columns={"attracion": "attraction"}, inplace=True)

MONTH_NAMES = list(calendar.month_name)
df["month_num"] = df.month.apply(lambda x: MONTH_NAMES.index(x))

In [ ]:
df.head()

In [ ]:
print(len(df.attraction.unique()), "unique attractions")
print("unique months: ", df.month.unique())
print(len(df), "datapoints, of which")
print("  ", df.status.value_counts()["opened"], "opened")
print("  ", df.status.value_counts()["closed"], "closed")
print(len(df.date.unique()), "unique dates")

## Visualize Missing Dates

Collect all dates for which at least one datapoint exists and group them by month and year. We can use this to visualize on which days the park was completely closed, e.g. due to Covid 19. 

In [ ]:
df_days = df[["month_num", "year", "date"]]
df_days = df_days.drop_duplicates()
df_days_agg = df_days.groupby(by=["year","month_num"]).aggregate("count")

for year in [2019, 2020, 2021]:
    for month in range(1, 13):
        try:
            df_days_agg.loc[(year, month), :]
        except KeyError:
            df_days_agg.loc[(year, month), :] = 0

In [ ]:
df_days_agg.sort_index().plot(kind="bar", figsize=(14, 6), ylabel="number of days", title="number of days per month for which at least one datapoint exists");

**suspicious months**
- 2019-11:
- 2020-06:

In [ ]:
df[["year", "month"]].drop_duplicates().groupby(by="month").agg("count").sort_values(by="year", ascending=False)

## Distribution of Datapoints per Day

In [ ]:
per_day_agg = df.groupby(by=["attraction", "date"]).agg("count")[["time"]]

In [ ]:
per_day_agg.head()

In [ ]:
df.head()

In [ ]:
# Show lines that are problematic because the timestamp is out of order
per_time_agg = df.groupby(by="time_min").agg("count")
per_time_agg[~(per_time_agg.index.str.endswith("0") | per_time_agg.index.str.endswith("5"))]

In [ ]:
FIX_TIMESTAMPS = {
    "09:11": "09:10", 
    "09:16": "09:15",
    "10:36": "10:35",
    "15:36": "15:35",
    "16:51": "16:50",
    "18:47": "18:45",
}
df_rounded_timestamps = df.replace(FIX_TIMESTAMPS)

In [ ]:
# Show lines that are problematic because the timestamp is out of order -> should be None after fixing
per_time_agg_rounded = df_rounded_timestamps.groupby(by="time_min").agg("count")
per_time_agg_rounded[~(per_time_agg_rounded.index.str.endswith("0") | per_time_agg_rounded.index.str.endswith("5"))]

In [ ]:
per_time_status_agg_rounded = df_rounded_timestamps.groupby(by=["time_min", "status"]).agg("count").reset_index()

In [ ]:
sns.barplot?

In [ ]:
sns.barplot(data=per_time_status_agg_rounded[per_time_status_agg_rounded.status == "opened"], x="time_min", y="wartezeit", color="green")
sns.barplot(data=per_time_status_agg_rounded[per_time_status_agg_rounded.status == "closed"], x="time_min", y="wartezeit", color="red")

In [ ]:
per_time_status_agg_rounded.status.unique()

In [ ]:
# timestamps range from 8:45 to 0:10 inclusive
print("expected number of different timestamps: ", (24-9)*(60/5) + 3 + 3)
print("actual number of different timestamps after rounding ", len(per_time_agg_rounded))

In [ ]:
per_time_agg_rounded.attraction.plot(xlabel="time rounded to nearest 5min", ylabel="number of attraction-day-combinations", 
    title="distribution of datapoints (regardless of open/closed) by time")

In [ ]:
df_agg_time_attraction = df_rounded_timestamps.groupby(by=["time_min", "attraction"]).agg("count").reset_index()
sns.lineplot(data=df_agg_time_attraction, x="time_min", y="wartezeit", hue="attraction")

# Quality by Months

- 19 months total, however February, March, April are missing completely
- months present in three years: July, August, September, October
- 